# CSC480 Project2: Simacogo
<p>Yuxuan Zhang</p>

<p>Human player goes first, using checker "O", play as MIN</p>
<p>Agent goes second, using checker "X", play as MAX.</p>

In [3]:
import numpy as np
import time

In [13]:
class Node(object):
    def __init__(self,state,action,depth,whos_move):
        self.state = state
        self.action = action # choice of column 1-9
        self.depth = depth # number of levels for the agent to search down the tree, should be an even number
        self.whos_move = whos_move # 1 if it's human's move, -1 if its agent's move
        self.best_score = None # best heuristic score for agent, which is agent score - human score, to be found later
        self.children = [] # will include children nodes
        self.best_child = None # the best child of the current node that maximize agent's gain
        #self.add_children() # populate its children when the node is created
    
    # return T/F whether the game is over, that is, whether the board is full
    def game_is_over(self):
        if np.sum(self.state=='-') == 0:
            return True
        else:
            return False
    
    # return score if the same neighbor is found
    def find_neighbor(self,row,col,checker,neighbor_type):
        try:
            if self.state[row,col] == checker:
                return 1*neighbor_type
            else:
                return 0
        except:
            return 0

    # get score for both agent and human
    # 2 points for every pair of pieces next to each other (up, down, left, right)
    # 1 point for every pair of pieces diagonally next to each other
    def get_score(self,checker):
        score = 0
        indices = np.nonzero(self.state==checker) # return row indices and column indices
        indices = zip(indices[0],indices[1]) # turn into (row,col) index pairs
        for row,col in indices:
            # up
            score += self.find_neighbor(row-1,col,checker,2)
            # left
            score += self.find_neighbor(row,col-1,checker,2)
            # down
            score += self.find_neighbor(row+1,col,checker,2)
            # right
            score += self.find_neighbor(row,col+1,checker,2)
            # up left
            score += self.find_neighbor(row-1,col-1,checker,1)
            # up right
            score += self.find_neighbor(row-1,col+1,checker,1)
            # down left
            score += self.find_neighbor(row+1,col-1,checker,1)
            # down right
            score += self.find_neighbor(row+1,col+1,checker,1)

        score /= 2 # remove the double count effect
        return score
                    
    def add_best_child(self,col):
        if self.depth > 0 and not self.game_is_over():
            row = len(self.state[:,col][self.state[:,col]=='-'])-1 # row index for dropping the checker
            if row >=0:
                new_state = self.state.copy()
                if self.whos_move == 1:
                    new_state[row,col] = 'X'
                else:
                    new_state[row,col] = 'O'
                self.best_child = Node(state=new_state,action=col,depth=self.depth-1,whos_move=-self.whos_move)

# given a node, grow the adversarial tree and return agent's action
def minimax(node, alpha, beta):
    #print 'node.depth:',node.depth,'node whos_move:',node.whos_move
    
    # when the search depth is reached or the game board is full, return agent score at terminal nodes
    if node.depth == 0 or node.game_is_over():
        # heurstic score is the difference between agent's and human's score
        agent_score = node.get_score('X')
        human_score = node.get_score('O')
        node.score = agent_score - human_score
        return node.score
    
    # when the upcoming is agent's move, which plays as MAX
    if node.whos_move == 1:
        agent_best_score = -10000
        
        # search for agent's moves
        for col in range(9):
            row = len(node.state[:,col][node.state[:,col]=='-'])-1 # row index for dropping the checker
            if row >=0:
                new_state = node.state.copy()
                new_state[row,col] = 'X'
                child = Node(state=new_state,action=col,depth=node.depth-1,whos_move=-node.whos_move)
                node.children.append(child) 
                temp_score = minimax(node=child, alpha=agent_best_score, beta=beta)
                if temp_score > agent_best_score:
                    agent_best_score = temp_score
                    node.best_score = agent_best_score
                    node.add_best_child(col)
                
                # pruning
                if agent_best_score > beta:
                    return agent_best_score
                    
                alpha = max(alpha,agent_best_score)
                #print 'Agent temp_score:',temp_score,'agent_best_score:',agent_best_score
        #print node.state
        #print 'Upcoming Agent move, depth:',node.depth,'whos_move next:',-node.whos_move,'best score for agent:',node.best_score,'best action for agent to choose:',node.best_child.action,'\n'
        return node.best_score
    
    # when the upcoming is human's move, which plays as MIN
    else:
        human_best_score = 10000
        
        # search for human's moves
        for col in range(9):
            row = len(node.state[:,col][node.state[:,col]=='-'])-1 # row index for dropping the checker
            if row >=0:
                new_state = node.state.copy()
                new_state[row,col] = 'O'
                child = Node(state=new_state,action=col,depth=node.depth-1,whos_move=-node.whos_move)
                node.children.append(child) 
                temp_score = minimax(node=child, alpha=alpha, beta=human_best_score)
                if temp_score < human_best_score:
                    human_best_score = temp_score
                    node.best_score = human_best_score
                    node.add_best_child(col)
                    
                # pruning
                if human_best_score < alpha:
                    return human_best_score
                    
                beta = min(beta,human_best_score)
                #print 'Human temp_score:',temp_score,'human_best_score:',human_best_score
        #print node.state
        #print 'Upcoming Human move, depth:',node.depth,'whos_move next:',-node.whos_move,'best score for human:',node.best_score,'best action for human to choose:',node.best_child.action,'\n'
        return node.best_score

In [24]:
init_state = np.zeros([9,9])
init_state=np.where(init_state==0.,'-',0)
init_state[8,[3,4]] = 'O'
init_state[7,3] = 'X'
init_state

array([['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['-', '-', '-', 'X', '-', '-', '-', '-', '-'],
       ['-', '-', '-', 'O', 'O', '-', '-', '-', '-']], 
      dtype='|S21')

In [28]:
# create a search tree based on current state
# action is the column that human player picked, assume to be 4
# depth should be an even number
# who's move=1 if upcoming is agent's turn to move, -1 if upcoming is human's turn
current_node = Node(state=init_state,action=4,depth=4,whos_move=1)

In [29]:
alpha = -10000 # best score for Agent, which wants to maximize it
beta = 10000 # best score for Human, which wants to minimize it

# return the best heuristic (score,action) for agent, action refers to which column to choose
expected_agent_score = minimax(node=current_node, alpha=alpha, beta=beta) 
print '\nNOW: expected_agent_score',expected_agent_score,'best_agent_action',current_node.best_child.action
print "State after agent's best move:"
print current_node.best_child.state


NOW: expected_agent_score -2 best_agent_action 4
State after agent's best move:
[['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' 'X' 'X' '-' '-' '-' '-']
 ['-' '-' '-' 'O' 'O' '-' '-' '-' '-']]
